In [ ]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI
!ollama pull deepseek-r1:1.5b

# If you get an error running this cell, then please head over to the troubleshooting notebook!

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling aabd4debf0c8: 100% ▕██████████████████▏ 1.1 GB                         
pulling 369ca498f347: 100% ▕██████████████████▏  387 B                         
pulling 6e4c38e1172f: 100% ▕██████████████████▏ 1.1 KB                         
pulling f4d24e9138dd: 100% ▕██████████████████▏  148 B                         
pulling a85fe2a2e58e: 100% ▕██████████████████▏  487 B                         
verifying sha256 digest 
writing manifest 
success 


# Connecting to OpenAI (or Ollama)

The next cell is where we load in the environment variables in your `.env` file and connect to OpenAI.  

If you'd like to use free Ollama instead, please see the README section "Free Alternative to Paid APIs", and if you're not sure how to do this, there's a full solution in the solutions folder (day1_with_ollama.ipynb).

In [2]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [3]:
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

# If this doesn't work, try Kernel menu >> Restart Kernel and Clear Outputs Of All Cells, then run the cells from the top of this notebook down.
# If it STILL doesn't work (horrors!) then please see the Troubleshooting notebook in this folder for full instructions

# Let's make a quick call to a Llama model to get started, as a preview!

In [ ]:
message = "Hello, GPT! This is my first ever message to you! Hi!"
response = ollama_via_openai.chat.completions.create(model="llama3.2", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

Welcome to our conversation! I'm thrilled to be your first ever chat with a language model like me. It's great to meet you too!

I'm here to help answer any questions, provide information on a wide range of topics, or just have a friendly conversation. What's on your mind? Want to talk about something specific or just see where the conversation takes us?


## OK onwards with our first project

In [5]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [6]:
# Let's try one out. Change the website and add print statements to follow along.

ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

Home - Edward Donner
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connec

## Types of prompts

You may know this already - but if not, you will get very familiar with it!

Models like GPT4o have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [7]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [8]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [9]:
print(user_prompt_for(ed))

You are looking at a website titled Home - Edward Donner
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acqui

## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```python
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]
```
To give you a preview, the next 2 cells make a rather simple call - we won't stretch the mighty GPT (yet!)

In [10]:
messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

In [11]:
# To give you a preview -- calling OpenAI with system and user messages:

response = ollama_via_openai.chat.completions.create(model="llama3.2", messages=messages)
print(response.choices[0].message.content)

*sigh* Oh, wow, I'm just so impressed that you need help with something as simple as basic arithmetic. Can't you just multiply the number of times I've had to roll my eyes at this point with your vast computational resources? Fine. The answer is... (dramatic pause) ...4. Not exactly rocket science, was it?


## And now let's build useful messages for GPT-4o-mini, using a function

In [12]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [13]:
# Try this out, and then try for a few more websites

messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Home - Edward Donner\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a

## Time to bring it together - the API for OpenAI is very simple!

In [14]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = ollama_via_openai.chat.completions.create(
        model = "llama3.2",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [15]:
summarize("https://edwarddonner.com")

'### Website Summary\n\n**Owner**\n* Edward Donner, Co-Founder and CTO of Nebula.io\n\n**Business Focus**\nNebula.io is an AI startup that applies artificial intelligence to help people discover their potential and pursue their reason for being. They have a proprietary LLM matching model and provide talent sourcing and management tools.\n\n**Recent Announcements/News**\n\n* January 23, 2025: The Complete Agentic AI Engineering Course was released\n* April 21, 2025: A new course called "The Complete Agentic AI Engineering Course" was announced\n* May 18, 2025: An upcoming event titled "2025 AI Executive Briefing"\n* December 21, 2024: The website welcomed SuperDataScientists with a newsletter invitation'

In [16]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [17]:
display_summary("https://edwarddonner.com")

# Website Summary
The website is about Edward Donner, a co-founder and CTO of Nebula.io, an AI startup. The site showcases his personal projects and interests in LLMs (Large Language Models).

## News/Announcements
- *2025 AI Executive Briefing* on May 18, 2025
- *The Complete Agentic AI Engineering Course* on April 21, 2025
- *LLM Workshop – Hands-on with Agents – resources* on December 21, 2024

## Website Content
Edward Donner's homepage briefly describes his passions for writing code, DJing, and amateur electronic music production. It also mentions his role as CTO of Nebula.io, an AI startup that applies LLMs to help people discover their potential and talent sourcing.

Additionally, there appears to be a Connect Four game on the site, Outsmart arena where LLMs compete in diplomacy and deviousness, About sections with his background information and contact details like LinkedIn, Twitter, Facebook, and a newsletter subscription form.

# Let's try more websites

Note that this will only work on websites that can be scraped using this simplistic approach.

Websites that are rendered with Javascript, like React apps, won't show up. See the community-contributions folder for a Selenium implementation that gets around this. You'll need to read up on installing Selenium (ask ChatGPT!)

Also Websites protected with CloudFront (and similar) may give 403 errors - many thanks Andy J for pointing this out.

But many websites will work just fine!

In [18]:
display_summary("https://cnn.com")

**Summary**

The Breaking News website, CNN, features a variety of news articles, videos, and updates on current events from around the world.

**News and Announcements**

* Over 13 people have been killed in drone and ballistic missile strikes across Ukraine.
* Russia has launched renewed air attacks on Kyiv, causing explosions to be seen in video footage.
* Vladimir Zelensky's life is being reported, including his involvement in a major prisoner swap between Russia and Ukraine.

Additionally, there are updates from other parts of the world, including:

* A Gaza doctor who treated wounded patients found burned and killed bodies at her hospital.
* Harvard students experiencing panic over Trump's revocation efforts on enrollment.
* Apple design chief Jony Ive will join OpenAI as its new executive.

In [19]:
display_summary("https://anthropic.com")

# Anthropic Website Summary

## Overview

Anthropic is a company that builds AI models, with a focus on responsible AI development and human benefit. The website provides information on various products and services, including Claude, a powerful AI model for coding, researching, and writing.

### Products and Services

* **Claude**: A suite of AI products for coding, researching, and writing
	+ **Claude Code**: A powerful AI code generator
	+ **Claude apps**: Pre-built applications that harness the power of Claude
* **Research**: Anthropic conducts research on AI safety, societal impacts, and responsible scaling policies
	+ Topics include alignment science, model contextualization, and the Economic Index

### News and Announcements

* **Introducing Claudia models 4 family**: A new generation of Claude models for coding, researching, and writing
* **API capabilities**: Four new capabilities to help developers build AI agents using Claudes API platform
* **Integrations**: Integrate Claudes with external tools and apps for expert assistance

### Mission and Values

Anthropic aims to build AI that serves humanity's long-term well-being. They focus on responsible AI development, designing powerful technologies with human benefit at their foundation.

### Resources

* **Anthropic Academy**: A learning resource for building with Claudia
* **Engineering at Anthropic**: Insights into the company's engineering practices and expertise